# Script that copies demand excel files from gdrive into bq tables

### imports and auth

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import re
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

# from google.colab import drive
# drive.mount('/content/drive')

### Declare Variables

In [ ]:
params = {
    'gdrive_folder_id' : '15JayKFd0LHqALJJ46MpcUyjWXNdS8pZB', # Toro templates drive folder
    'local_download_path' : '/content/data', # the path to download the gdrive data to e.g. '/content/data'
    'sheet_names' : ['HW','TRUG','BUN'], # These are the sheet names for each spreadsheet to read in as pandas dataframe
}

### Get data from gdrive

In [ ]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser(params['local_download_path'])
try:
  os.makedirs(local_download_path)
except: pass


### Get file ID's to download >year>mth>wk>demand planning folder>file

In [ ]:
def get_drive_folder_content(drive_id):
  return drive.ListFile({'q': f"'{drive_id}' in parents"}).GetList()

def get_match_id(drive_id, regex='.*'):
  import re
  l = []
  drive_list = get_drive_folder_content(drive_id=drive_id)
  for d in drive_list:
    d_match = re.match(regex, d['title'])
    if d_match:
      l.append(d['id'])
  return l

def get_match_title(drive_id, regex='.*'):
  import re
  l = []
  drive_list = get_drive_folder_content(drive_id=drive_id)
  for d in drive_list:
    d_match = re.match(regex, d['title'])
    if d_match:
      l.append(d['title'])
  return l


In [ ]:
year_match = get_match_id(drive_id=params['gdrive_folder_id'], regex='^2021$|^2022$|^2023$')
month_match = []
for x in year_match:
  month_match.append(get_match_id(drive_id=x, regex='.*'))

week_match = []
for x in month_match:
  for iv in x:
    week_match.append(get_match_id(drive_id=iv, regex='.*'))

demandPlanning_folders = []
for x in week_match:
  for iv in x:
    demandPlanning_folders.append(get_match_id(drive_id=iv, regex='^Demand Planning$'))

files_2_dl = []
files_2_dl_fnames = []
for x in demandPlanning_folders:
  for iv in x:
    files_2_dl.append(get_match_id(drive_id=iv, regex='^35 week Primal Advice [0-9]{2}-[0-9]{2}-[0-9]{4} .*xlsx$'))
    files_2_dl_fnames.append(get_match_title(drive_id=iv, regex='^35 week Primal Advice [0-9]{2}-[0-9]{2}-[0-9]{4} .*xlsx$'))

flatten_files_2_dl = [item for sublist in files_2_dl for item in sublist]
flatten_files_2_dl_fnames = [item for sublist in files_2_dl_fnames for item in sublist]

for t,i in zip(flatten_files_2_dl_fnames,flatten_files_2_dl):
  # 3. Create & download by id.
  print('title: %s, id: %s' % (t, i))
  fname = os.path.join(local_download_path, t)
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': i})
  f_.GetContentFile(fname)

title: 35 week Primal Advice 23-03-2022 with unlimited capasity (1).xlsx, id: 1vq1J-nxG2oBe8mZYC96cZb4LZf10xeui
downloading to /content/data/35 week Primal Advice 23-03-2022 with unlimited capasity (1).xlsx
title: 35 week Primal Advice 19-03-2022 with unlimited capasity.xlsx, id: 1hxmmEssH0JzjbutCj8y1jkufuA6u-619
downloading to /content/data/35 week Primal Advice 19-03-2022 with unlimited capasity.xlsx
title: 35 week Primal Advice 11-03-2022 with unlimited capasity.xlsx, id: 1hBhRhiJ2JvHDLo_3uVgsK66Y5pvUlE3y
downloading to /content/data/35 week Primal Advice 11-03-2022 with unlimited capasity.xlsx
title: 35 week Primal Advice 03-03-2022 with unlimited capasity.xlsx, id: 1RBzUuA0KX-xkBpCCnCbZW2OJHgejD-2f
downloading to /content/data/35 week Primal Advice 03-03-2022 with unlimited capasity.xlsx
title: 35 week Primal Advice 25-02-2022 with unlimited capasity.xlsx, id: 1YqryttwDQiZguGUylLSTY_Xmh5s9_d0S
downloading to /content/data/35 week Primal Advice 25-02-2022 with unlimited capasity.xl

### from excel to pandas dataframes

In [ ]:
df_dict = {}
for f in os.listdir(params['local_download_path']):
  f_path = params['local_download_path']+'/'+f
  df_dict[f] = pd.read_excel(f_path, sheet_name=params['sheet_names'])


### Add columns to each dataframe

In [ ]:
#df_dict['35 week Primal Advice 09-09-2021 with unlimited capasity.xlsx']#['HW']#.describe()
# for each sheet of data in excel file (now dict of dataframes in dict where key = filename and subkey = sheetName)
# add a column for sheet name e.g. HW, TRUG, BUN

def get_dmy_from_filename(fn):
  title_search = re.search('([0-9]{2})-([0-9]{2})-([0-9]{4})', fn, re.IGNORECASE)
  if title_search:
    return title_search.group(3)+'-'+title_search.group(2)+'-'+title_search.group(1)
  else:
    return ''

for ss in df_dict.keys():
  for sname in df_dict[ss].keys():
    df_dict[ss][sname]['sheet_name'] = sname
    df_dict[ss][sname]['source_date'] = get_dmy_from_filename(ss)
    df_dict[ss][sname]['file_name'] = ss


### make single dataframe

In [ ]:
def stack_df(dict_of_df):
  return pd.concat(dict_of_df).reset_index().drop(columns=['level_0','level_1'])

all_df = pd.concat([stack_df(df_dict[ss]) for ss in df_dict.keys()])

### clean column names

In [ ]:
all_df.columns = all_df.columns.str.replace(' ','_', regex=False)
all_df.columns = all_df.columns.str.replace('.','_', regex=False)

In [ ]:
all_df.columns

### reorder columns

In [ ]:
# Stefan Petiq 20220214:
# Everything exactly how it is except
# Wow_code
# Plant
# Sheet_name
# Source_date
# file_name
# moved to be the last 5 fields at the end of the sheet

last_cols = ['WOW_code', 'Plant', 'sheet_name', 'source_date', 'file_name']
new_col_order = [x for x in all_df.columns if x not in last_cols]+last_cols

all_df = all_df[new_col_order]

### Delete unwanted sheet

In [ ]:
# Delete 'week' from the "35 week Primal Advice 10-01-2022 with unlimited capasity.xlsx" file so it should not appear in the data loading anymore
all_df = all_df.query("file_name != '35 week Primal Advice 10-01-2022 with unlimited capasity.xlsx'")

### change dtypes

In [ ]:
all_df.columns

In [ ]:
all_df.drop(columns='week', inplace=True)

### save to bigquery

In [ ]:
def float2str(d):
  return ['' if np.isnan(x) else str(int(x)) for x in d]
all_df['WOW_code'] = float2str(all_df['WOW_code'])
all_df['PrimalID'] = float2str(all_df['PrimalID'])

In [ ]:
pd.io.gbq.to_gbq(all_df, 'hilton.hilton_primaladvice_35weeks', 'gcp-wow-pvc-grnstck-prod', chunksize=100000, reauth=False, if_exists='replace')